In [77]:
import workflow.scripts.resources as res_module
import pandas as pd
import geopandas as gpd

In [78]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis
store=f"data/store/resources_AB.h5"
datahandler=DataHandler(store)

# Set Required Args to Activate Modules

In [ ]:
# Iterate over provinces for both solar and wind resources
resource_types = ['wind'] # , 'solar'
provinces=['AB']  #'AB','SK','ON','NS'
for province_code in provinces:
    for resource_type in resource_types:
        required_args = {
            "config_file_path": 'config/config.yaml',
            "province_short_code": province_code,
            "resource_type": resource_type
        }
        
        # Create an instance of Resources and execute the module
        resources = res_module.Resources(**required_args)
        resources.execute_module()

2024-12-10 10:43:02,865 - INFO - Preparing Grid Cells...
2024-12-10 10:43:02,865 - INFO - >> Loading GADM boundaries (Sub-provincial | level =2) for Alberta  from local file data/processed_data/regions/gadm41_Canada_L2_AB.geojson.
2024-12-10 10:43:02,913 - INFO - Cutout already prepared.
2024-12-10 10:43:03,055 - INFO - >> Loading GADM boundaries (Sub-provincial | level =2) for Alberta  from local file data/processed_data/regions/gadm41_Canada_L2_AB.geojson.


__________________________________________________
 Initiating wind module for Alberta...

    >>> Memory management remarks:
    * After execution, all downloaded data is stored at cutout.path. By default, it is not loaded into memory, but into dask arrays. This keeps the memory consumption extremely low.
    * The data is accessible in cutout.data, which is an xarray.Dataset. Querying the cutout gives us some basic information on which data is contained in it.
    * For more operations related to cutout, check the tool docs @ https://atlite.readthedocs.io/en/master/examples/create_cutout.html#
        
>> Updated 'cells' saved to data/store/resources_AB.h5 with key 'cells'
>> Updated 'boundary' saved to data/store/resources_AB.h5 with key 'boundary'


2024-12-10 10:43:03,083 - INFO - Cutout already prepared.
2024-12-10 10:43:03,083 - INFO - >> Loading global filters' rasters from GAEZ, trimmed to Alberta
2024-12-10 10:43:03,084 - INFO - >> Raster file 'faocmb_2010.tif' already exists locally.
2024-12-10 10:43:03,084 - INFO - >> Raster file 'slpmed05.tif' already exists locally.
2024-12-10 10:43:03,084 - INFO - >> Raster file 'exclusion_2017.tif' already exists locally.
2024-12-10 10:43:03,085 - INFO - >> Loading GADM boundaries (Sub-provincial | level =2) for Alberta  from local file data/processed_data/regions/gadm41_Canada_L2_AB.geojson.
2024-12-10 10:43:03,196 - INFO - >> Raster plot saved at: vis/misc/faocmb_2010_raster_AB.png



    >>> Memory management remarks:
    * After execution, all downloaded data is stored at cutout.path. By default, it is not loaded into memory, but into dask arrays. This keeps the memory consumption extremely low.
    * The data is accessible in cutout.data, which is an xarray.Dataset. Querying the cutout gives us some basic information on which data is contained in it.
    * For more operations related to cutout, check the tool docs @ https://atlite.readthedocs.io/en/master/examples/create_cutout.html#
        


2024-12-10 10:43:03,291 - INFO - >> Raster plot saved at: vis/misc/slpmed05_raster_AB.png
2024-12-10 10:43:03,468 - INFO - >> Raster plot saved at: vis/misc/exclusion_2017_raster_AB.png
2024-12-10 10:43:03,469 - INFO - All required rasters for GAEZ processed and plotted successfully.
2024-12-10 10:43:03,469 - INFO -  >>> Loading Land_cover layers from data/downloaded_data/GAEZ/Rasters_in_use/LR/lco/AB_faocmb_2010.tif
2024-12-10 10:43:03,469 - INFO -  >>> Loading Terrain_resources layers from data/downloaded_data/GAEZ/Rasters_in_use/LR/ter/AB_slpmed05.tif
2024-12-10 10:43:03,469 - INFO -  >>> Loading Exclusion_areas layers from data/downloaded_data/GAEZ/Rasters_in_use/LR/excl/AB_exclusion_2017.tif
2024-12-10 10:43:03,470 - INFO - > Loading Canadian Protected and Conserved Areas Database (CPCAD) from locally stored datafile - data/downloaded_data/lands/ProtectedConservedArea_AB.pickle
2024-12-10 10:43:03,479 - INFO - >> Loading locally stored OSM data for 'aeroway' from data/downloaded_d

# Temp tests

In [ ]:
CPCAD=pd.read_pickle('data/downloaded_data/lands/ProtectedConservedArea.pickle')

In [ ]:
datahandler.show_tree(store)

In [ ]:
boundary=datahandler.from_store('boundary')

In [ ]:
CPCAD=pd.read_pickle('data/downloaded_data/lands/ProtectedConservedArea_AB.pickle')

In [ ]:
CPCAD.explore('NAME_E')

In [ ]:
# Filter for Banff and Waterton Lakes National Parks
banff_and_waterton = CPCAD[
    (CPCAD['NAME_E'] == 'Banff National Park Of Canada') | 
    (CPCAD['NAME_E'] == 'Waterton Lakes National Park Of Canada')
]

# Check CRS and reproject if needed
if banff_and_waterton.crs.is_geographic:
    banff_and_waterton = banff_and_waterton.to_crs(epsg=3857)  # Reproject to meters for accurate buffering

# Apply the buffer (e.g., 10 km buffer)
buffered_boundary = banff_and_waterton.geometry.buffer(30000)  # Distance in meters

# Create a new GeoDataFrame with the buffered geometry
buffered_banff = gpd.GeoDataFrame(banff_and_waterton, geometry=buffered_boundary)

# Reproject back to the original CRS if needed
buffered_banff = buffered_banff.to_crs(CPCAD.crs)

# Plot the buffered area
buffered_banff.explore()


In [ ]:
datahandler.show_tree(store)

In [ ]:
cells=datahandler.from_store('cells')
cells.to_pickle('AB_temp_analysis/Cells_without_regulatory_buffer.pkl')

In [ ]:
cells.columns

In [ ]:
cells.explore('potential_capacity_wind',cmap='Blues',alpha=0.6)